In [1]:
from os.path import dirname, join, pardir

#from cobra.io import read_sbml_model

from cobra.io import save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
from dfba import DfbaModel, ExchangeFlux, KineticVariable

import time
import cobra

start_time = time.time()

In [36]:
# DfbaModel instance initialized with cobra model
small_model = read_sbml_model('/home/csiharath/Téléchargements/model_cancer_noDrugs.xml')
small_model.solver = "glpk"

Adding exchange reaction EX_ALA with default bounds for boundary metabolite: ALA.
Adding exchange reaction EX_ARG with default bounds for boundary metabolite: ARG.
Adding exchange reaction EX_ASN with default bounds for boundary metabolite: ASN.
Adding exchange reaction EX_ASP with default bounds for boundary metabolite: ASP.
Adding exchange reaction EX_EGLC with default bounds for boundary metabolite: EGLC.
Adding exchange reaction EX_EGLN with default bounds for boundary metabolite: EGLN.
Adding exchange reaction EX_EGLU with default bounds for boundary metabolite: EGLU.
Adding exchange reaction EX_GLY with default bounds for boundary metabolite: GLY.
Adding exchange reaction EX_HIS with default bounds for boundary metabolite: HIS.
Adding exchange reaction EX_ILE with default bounds for boundary metabolite: ILE.
Adding exchange reaction EX_LAC with default bounds for boundary metabolite: LAC.
Adding exchange reaction EX_LYS with default bounds for boundary metabolite: LYS.
Adding exc

In [37]:
small_model.objective = 'Vgrowth'
small_model.summary()

--- Logging error ---
Traceback (most recent call last):
  File "/home/csiharath/anaconda3/lib/python3.8/logging/__init__.py", line 1088, in emit
    stream.write(msg + self.terminator)
  File "/home/csiharath/anaconda3/lib/python3.8/tempfile.py", line 473, in func_wrapper
    return func(*args, **kwargs)
ValueError: I/O operation on closed file.
Call stack:
  File "/home/csiharath/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/csiharath/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/csiharath/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/csiharath/anaconda3/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/home/csiharath/anaconda3/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 677, in start
    sel

Metabolite,Reaction,Flux,C-Number,C-Flux
ARG,EX_ARG,26.85,0,0.00%
ASN,EX_ASN,26.85,0,0.00%
ASP,EX_ASP,20.13,0,0.00%
ATP,EX_ATP,1000,0,0.00%
EGLC,EX_EGLC,541.9,0,0.00%
EGLN,EX_EGLN,526.8,0,0.00%
GLY,EX_GLY,40.27,0,0.00%
HIS,EX_HIS,13.42,0,0.00%
ILE,EX_ILE,60.4,0,0.00%
LYS,EX_LYS,40.27,0,0.00%


In [38]:
small_model.reactions.Vgrowth

Reaction identifier,Vgrowth
Name,Vgrowth
Memory address,0x07fd2ff695520
Stoichiometry,0.06 ALA + 0.04 ARG + 0.04 ASN + 0.03 ASP + 3.78 ATP + 0.02 EGLN + 0.04 EGLU + 0.03 G6P + 0.06 GLY + 0.02 HIS + 0.09 ILE + 0.06 LYS + 0.01 MET + 0.09 PALM + 0.02 PHE + 0.03 PRO + 0.03 R5P + 0.05... 0.06 ALA + 0.04 ARG + 0.04 ASN + 0.03 ASP + 3.78 ATP + 0.02 EGLN + 0.04 EGLU + 0.03 G6P + 0.06 GLY + 0.02 HIS + 0.09 ILE + 0.06 LYS + 0.01 MET + 0.09 PALM + 0.02 PHE + 0.03 PRO + 0.03 R5P + 0.05...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [39]:
small_model.reactions.get_by_id("VPFK").lower_bound = 0
small_model.reactions.get_by_id("VPFK").upper_bound = 1000
# G6PDH
small_model.reactions.get_by_id("VG6PDH").lower_bound = 0
small_model.reactions.get_by_id("VG6PDH").upper_bound = 1000
# TKT
small_model.reactions.get_by_id("VTK").lower_bound = 0
small_model.reactions.get_by_id("VTK").upper_bound = 1000
# PALM
small_model.reactions.get_by_id("VPALM").lower_bound = 0
small_model.reactions.get_by_id("VPALM").upper_bound = 1000


In [40]:
from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(small_model)

{<Reaction Vgrowth at 0x7fd2ff695520>: 1.0}

In [41]:
dfba_model = DfbaModel(small_model)
dfba_model.add_objectives(["Vgrowth"], ["max"])
#dfba_model.add_objectives("Vgrowth","max")
print(dfba_model.objectives)

['Vgrowth']


In [42]:
# instances of KineticVariable (default initial conditions are 0.0, but can be
# set here if wanted e.g. Oxygen)
X = KineticVariable("Biomass", initial_condition=1.04e-4)
Gluc = KineticVariable("EGLC")
Lac = KineticVariable("LAC")
Gln = KineticVariable("EGLN")
Amp = KineticVariable("AMP")
Pyr = KineticVariable("PYR")
Palm = KineticVariable("PALM")
AcoA = KineticVariable("ACCOA")

In [43]:
# add kinetic variables to dfba_model
dfba_model.add_kinetic_variables([X, Gluc, Lac, Gln, Amp, Pyr, Palm, AcoA])

In [44]:
# instances of ExchangeFlux
mu = ExchangeFlux("Vgrowth")
v_G = ExchangeFlux("EX_EGLC") # glucose[x] <=> glucose[s]  HMR_9034
v_N = ExchangeFlux("EX_EGLN") # EGLN[x] <=> EGLN[s] HMR_9063
v_L = ExchangeFlux("VLDH")
v_D = ExchangeFlux("VPPRibP")
v_P = ExchangeFlux("VPALM")


In [45]:

# add exchange fluxes to dfba_model
dfba_model.add_exchange_fluxes([mu, v_G, v_L, v_N, v_D, v_P])


In [46]:
# add rhs expressions for kinetic variables in dfba_model
dfba_model.add_rhs_expression("Biomass", mu * X)
dfba_model.add_rhs_expression("EGLC", -v_G * 16.0 * X / 1000.0)
dfba_model.add_rhs_expression("EGLN", -v_N * 16.0 * X / 1000.0)
dfba_model.add_rhs_expression("LAC", v_L * 150.13 * X / 1000.0)
dfba_model.add_rhs_expression("AMP", v_D * 150.13 * X / 1000.0)
dfba_model.add_rhs_expression("PALM", v_P * 150.13 * X / 1000.0)
dfba_model.add_rhs_expression("PYR", v_P * 150.13 * X / 1000.0)
dfba_model.add_rhs_expression("ACCOA", v_P * 150.13 * X / 1000.0)



In [47]:
# add lower/upper bound expressions for exchange fluxes in dfba_model together
# with expression that must be non-negative for correct evaluation of bounds
# dfba_model.add_exchange_flux_lb(
#     "GLCex", 10.5 * (Gluc / (0.0027 + Gluc)) * (1 / (1 + Eth / 20.0)), Gluc
# )
dfba_model.add_exchange_flux_lb("EX_EGLC", 10.5 * (Gluc / (0.0027 + Gluc)), Gluc)
dfba_model.add_exchange_flux_lb("EX_EGLN",6.0 * (Gln / (0.0165 + Gln)) * (1 / (1 + Gluc / 0.005)),Gln)
dfba_model.add_exchange_flux_lb("VPALM", (2.95e-5 * (AcoA/(1e-2 + AcoA)) - Palm), Palm)#  * ((Atp/Adp)/(5 * (Atp/Adp))) * ((Nadph/Nadp)/(0.5 * (Nadph/Nadp))))\

In [48]:
print(dfba_model.kinetic_variables)

[yval[0], yval[1], yval[2], yval[3], yval[4], yval[5], yval[6], yval[7]]


In [49]:
# add initial conditions for kinetic variables in dfba_model X (gDW/L),
# metabolites (g/L)
dfba_model.add_initial_conditions(
    {
        "Biomass": 1.04e-4,
        "EGLC": 1,
        "EGLN": 0,
        "LAC": 0,
        "PALM": 0
    }
)

In [50]:
# simulate model across interval t = [0.0,25.0](hours) with outputs for plotting
# every 0.1h and optional list of fluxes
concentrations, trajectories = dfba_model.simulate(
    0.0, 25, 0.1, ["Vgrowth", "EX_EGLC"] # "VLDH", "VPPRibP", "VPALM"
)

cc1plus: warning: command line option '-Wstrict-prototypes' is valid for C/ObjC but not for C++


Setting objective 0:  column 127 coefficient 1 column 128 coefficient -1 direction 2
GLPK Simplex Optimizer, v4.65
48 rows, 132 columns, 426 non-zeros
glp_simplex: initial basis is invalid

Total simulation time was 0.00075 seconds



In [51]:
from dfba.plot.plotly import *

import plotly.io as pio

pio.templates.default = "plotly_white"

fig = plot_concentrations(concentrations)
fig.show()

fig = plot_trajectories(trajectories)
fig.show()